In [1]:
# -*- coding: utf-8 -*-
import numpy as np
from scipy import sparse
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

from grad_utils import grad_logloss_theta_lr
from grad_utils import batch_grad_logloss_lr
from inverse_hvp import inverse_hvp_lr_newtonCG
from dataset import load_data_v1,select_from_one_class

import argparse
import time
import pdb
import os

In [2]:
# select the dataset used
dataset_name = "cancer"
# parameter for the sigmoid sampling function
sigmoid_k = 10
# regularization parameter for Logistic Regression
C = 0.1
# sample ratio
sample_ratio = 0.95

In [3]:
start_time = time.time()
# load data, pick 30% as the Va set
x_train,y_train,x_va,y_va,x_te,y_te = load_data_v1(dataset_name,va_ratio=0.3)
print("x_train, nr sample {}, nr feature {}".format(x_train.shape[0],x_train.shape[1]))
print("x_va,    nr sample {}, nr feature {}".format(x_va.shape[0],x_va.shape[1]))
print("x_te,    nr sample {}, nr feature {}".format(x_te.shape[0],x_te.shape[1]))
print("Tr: Pos {} Neg {}".format(y_train[y_train==1].shape[0],y_train[y_train==0].shape[0]))
print("Va: Pos {} Neg {}".format(y_va[y_va==1].shape[0],y_va[y_va==0].shape[0]))
print("Te: Pos {} Neg {}".format(y_te[y_te==1].shape[0],y_te[y_te==0].shape[0]))
print("Load data, cost {:.1f} sec".format(time.time()-start_time))

x_train, nr sample 350, nr feature 11
x_va,    nr sample 150, nr feature 11
x_te,    nr sample 183, nr feature 11
Tr: Pos 164 Neg 186
Va: Pos 33 Neg 117
Te: Pos 42 Neg 141
Load data, cost 0.0 sec


In [4]:
# get the subset samples number
num_tr_sample = x_train.shape[0]
obj_sample_size = int(sample_ratio * num_tr_sample)

# define the full-set-model \hat{\theta}
clf = LogisticRegression(
        C = C,
        fit_intercept=False,
        tol = 1e-8,
        solver="liblinear",
        multi_class="ovr",
        max_iter=100,
        warm_start=False,
        verbose=0,
        )
clf.fit(x_train,y_train)
# on Va
y_va_pred = clf.predict_proba(x_va)[:,1]
full_logloss = log_loss(y_va,y_va_pred)
weight_ar = clf.coef_.flatten()
# on Te
y_te_pred = clf.predict_proba(x_te)
full_te_logloss = log_loss(y_te,y_te_pred)

# print full-set-model results
print("[FullSet] Va logloss {:.6f}".format(full_logloss))
print("[FullSet] Te logloss {:.6f}".format(full_te_logloss))


[FullSet] Va logloss 0.105347
[FullSet] Te logloss 0.091384


In [5]:
# get time cost for computing the IF
if_start_time = time.time()
# building precoditioner
test_grad_loss_val = grad_logloss_theta_lr(y_va,y_va_pred,x_va,weight_ar,C,False,0.1/(num_tr_sample*C))
tr_pred = clf.predict_proba(x_train)[:,1]
batch_size = 10000
M = None
total_batch = int(np.ceil(num_tr_sample / float(batch_size)))
for idx in range(total_batch):
    batch_tr_grad = batch_grad_logloss_lr(y_train[idx*batch_size:(idx+1)*batch_size],
        tr_pred[idx*batch_size:(idx+1)*batch_size],
        x_train[idx*batch_size:(idx+1)*batch_size],
        weight_ar,
        C,
        False,
        1.0)

    sum_grad = batch_tr_grad.multiply(x_train[idx*batch_size:(idx+1)*batch_size]).sum(0)
    if M is None:
        M = sum_grad
    else:
        M = M + sum_grad

M = M + 0.1/(num_tr_sample*C) * np.ones(x_train.shape[1])
M = np.array(M).flatten()
# computing the inverse Hessian-vector-product
iv_hvp = inverse_hvp_lr_newtonCG(x_train,y_train,tr_pred,test_grad_loss_val,C,True,1e-5,True,M,0.1/(num_tr_sample*C))
# get influence score
total_batch = int(np.ceil(x_train.shape[0] / float(batch_size)))
predicted_loss_diff = []
for idx in range(total_batch):
    train_grad_loss_val = batch_grad_logloss_lr(y_train[idx*batch_size:(idx+1)*batch_size],
        tr_pred[idx*batch_size:(idx+1)*batch_size],
        x_train[idx*batch_size:(idx+1)*batch_size],
        weight_ar,
        C,
        False,
        1.0)

    predicted_loss_diff.extend(np.array(train_grad_loss_val.dot(iv_hvp)).flatten())
    
predicted_loss_diffs = np.asarray(predicted_loss_diff)
duration = time.time() - if_start_time
print("The Influence function's computation completed, cost {:.1f} sec".format(duration))

Succeed in getting the inverse of preconditioner M.
iter 0 cg iter 0 iter_diff 0.017279105578539244
Function value: -0.0006547602177645558
Split function value: 0.0006547580515490153, -0.0013095182693135712
iter 1 cg iter 0 iter_diff 0.0009571376369245043
Function value: -0.0006715980575009302
Split function value: 0.0006715611449095135, -0.0013431592024104437
iter 2 cg iter 0 iter_diff 4.758424530440638e-06
Function value: -0.0006715983390175697
Split function value: 0.0006716115759386227, -0.0013432099149561923
iter 3 cg iter 0 iter_diff 7.491056926911197e-07
Function value: -0.0006715983390175697
Split function value: 0.0006716115759386227, -0.0013432099149561923
Optimization terminated successfully.
         Current function value: -0.000672
         Iterations: 4
         Function evaluations: 5
         Gradient evaluations: 8
         Hessian evaluations: 9
implicit hessian-vector products mean: -0.06288588997311396
implicit hessian-vector products norm: 0.30488225907448274
Inve

D:\subsampling\code_release\inverse_hvp.py:151: OptimizeWarning: Unknown solver options: preconditioner
  preconditioner=M)


In [6]:
print("=="*30)
print("IF Stats: mean {:.10f}, max {:.10f}, min {:.10f}".format(
    predicted_loss_diffs.mean(), predicted_loss_diffs.max(), predicted_loss_diffs.min())
)
# build sampling probability
phi_ar = - predicted_loss_diffs
IF_interval = phi_ar.max() - phi_ar.min()
a_param = sigmoid_k / IF_interval
prob_pi = 1 / (1 + np.exp(a_param * phi_ar))
print("Pi Stats:",np.percentile(prob_pi,[10,25,50,75,90]))
# Do subsampling
pos_idx = select_from_one_class(y_train,prob_pi,1,sample_ratio)
neg_idx = select_from_one_class(y_train,prob_pi,0,sample_ratio)
sb_idx = np.union1d(pos_idx,neg_idx)
sb_x_train = x_train[sb_idx]
sb_y_train = y_train[sb_idx]

# Train the subset-model \tilde{\theta}
clf.fit(sb_x_train,sb_y_train)
y_va_pred = clf.predict_proba(x_va)[:,1]
sb_logloss = log_loss(y_va, y_va_pred)
sb_weight = clf.coef_.flatten()
diff_w_norm = np.linalg.norm(weight_ar - sb_weight)
sb_size = sb_x_train.shape[0]
y_te_pred = clf.predict_proba(x_te)[:,1]
sb_te_logloss = log_loss(y_te,y_te_pred)


# baseline: random sampling
u_idxs = np.arange(x_train.shape[0])
uniform_idxs = np.random.choice(u_idxs,obj_sample_size,replace=False)
us_x_train = x_train[uniform_idxs]
us_y_train = y_train[uniform_idxs]
clf.fit(us_x_train, us_y_train)
y_va_pred = clf.predict_proba(x_va)[:,1]
us_logloss = log_loss(y_va, y_va_pred)
us_size = us_x_train.shape[0]
y_te_pred = clf.predict_proba(x_te)[:,1]
us_te_logloss = log_loss(y_te,y_te_pred)

print("=="*30)
print("Result Summary on Va")
print("[SigUIDS]  logloss {:.6f}, # {}".format(sb_logloss,sb_size))
print("[Random]   logloss {:.6f}, # {}".format(us_logloss,us_size))
print("[Full]     logloss {:.6f}, # {}".format(full_logloss,num_tr_sample))
print("Result Summary on Te")
print("[SigUIDS]  logloss {:.6f}, # {}".format(sb_te_logloss,sb_size))
print("[Random]   logloss {:.6f}, # {}".format(us_te_logloss,us_size))
print("[Full]     logloss {:.6f}, # {}".format(full_te_logloss,num_tr_sample))
print("=="*30)
# Attention: if the dataset used here is small, one experiment may fail because of uncertainty of subsampling!
# besides, a proper $k$ for sigmoid sampling function is also important.


IF Stats: mean 0.0011671405, max 0.0107398559, min -0.0405537893
Pi Stats: [0.44256842 0.53599139 0.60420381 0.65264003 0.7068804 ]
Result Summary on Va
[SigUIDS]  logloss 0.094405, # 331
[Random]   logloss 0.103259, # 332
[Full]     logloss 0.105347, # 350
Result Summary on Te
[SigUIDS]  logloss 0.082369, # 331
[Random]   logloss 0.089804, # 332
[Full]     logloss 0.091384, # 350
